In [24]:
import pandas as pd
from sqlalchemy import create_engine
import plotly.express as px

In [186]:
engine = create_engine('sqlite:////Users/paz/Documents/github/home-property-tax/wickersham/tax_year_2023/database/home_property_tax.db')
zwickersham_acct = '931920000058'

# The Market

In [217]:
market_trends_query = 'select * from real_market_yearly'

In [218]:
market_trends_df = pd.read_sql_query(market_trends_query, engine)

In [219]:
fig = px.line(market_trends_df, x='year', y=['subject_mkt_value','nbhd_grp_mkt_value','hcad_a1_mkt_value'], template='simple_white')
fig.show()

In [220]:
fig = px.line(market_trends_df, x='year', y=['subject_mkt_value_yoy','nbhd_grp_mkt_value_yoy','hcad_a1_mkt_value_yoy'], template='simple_white')
fig.show()

In [221]:
market_trends_df[['nbhd_grp_mkt_value_yoy', 'hcad_a1_mkt_value_yoy', 'subject_mkt_value_yoy']].corr(method='pearson')

,nbhd_grp_mkt_value_yoy,hcad_a1_mkt_value_yoy,subject_mkt_value_yoy
nbhd_grp_mkt_value_yoy,1.00000,0.719070,0.612470
hcad_a1_mkt_value_yoy,0.71907,1.000000,0.748584
subject_mkt_value_yoy,0.61247,0.748584,1.000000


In [222]:
market_trends_df

,year,subject_mkt_value,subject_prior_mkt_value,subject_mkt_value_yoy,legal_descr_mkt_value,legal_descr_prior_mkt_value,legal_desc_mkt_value_yoy,nbhd_cd_mkt_value,nbhd_cd_prior_mkt_value,nbhd_cd_mkt_value_yoy,nbhd_grp_mkt_value,nbhd_prior_grp_mkt_value,nbhd_grp_mkt_value_yoy,map_facet_mkt_value,map_facet_prior_mkt_value,map_facet_mkt_value_yoy,hcad_a1_mkt_value,hcad_a1_mkt_prior_value,hcad_a1_mkt_value_yoy
0,2005,195500.0,195500.0,0.000000,298292.0,288850.0,0.032688,258800.0,259550.0,-0.002890,292900.0,287150.0,NaN,156000.0,157050.0,-0.006686,111480.0,105600.0,0.055682
1,2006,217000.0,195500.0,0.109974,300000.0,298264.0,0.005820,279805.0,258800.0,0.081163,303694.5,292900.0,NaN,163764.5,154800.0,0.057910,115372.0,109578.0,0.052876
2,2007,220000.0,217000.0,0.013825,315961.0,300000.0,0.053203,284075.0,279805.0,0.015261,315000.0,303694.5,NaN,169623.0,162360.0,0.044734,121530.5,113678.0,0.069077
3,2008,230157.0,220000.0,0.046168,338900.0,316000.0,0.072468,299273.0,284048.0,0.053600,337043.5,315000.0,NaN,175386.0,167439.0,0.047462,124785.0,120934.0,0.031844
4,2009,224000.0,230157.0,-0.026751,351000.0,338821.5,0.035944,301922.0,299136.5,0.009312,348104.5,336970.5,NaN,176032.5,175253.5,0.004445,118909.5,123928.0,-0.040495
5,2010,207172.0,224000.0,-0.075125,337468.5,351000.0,-0.038551,292493.0,301922.0,-0.031230,333923.5,348104.5,-0.045114,173217.0,176032.5,-0.015994,116121.0,118223.0,-0.017780
6,2011,207172.0,207172.0,0.000000,337026.5,337400.0,-0.001107,292196.0,292493.0,-0.001015,333478.5,333536.5,-0.004542,162291.0,173133.0,-0.062622,114403.0,115537.0,-0.009815
7,2012,207172.0,207172.0,0.000000,338219.0,336910.0,0.003885,291553.0,292292.0,-0.002528,333567.0,333470.0,-0.004273,160131.0,162291.0,-0.013309,110961.0,113822.0,-0.025136
8,2013,245000.0,207172.0,0.182592,366928.0,338000.0,0.085586,319223.0,291474.5,0.095200,365962.0,335000.0,0.092424,161469.0,160154.0,0.008211,112948.0,110028.0,0.026539
9,2014,292216.0,245000.0,0.192718,444268.0,366894.0,0.210889,389821.5,319223.0,0.221157,442343.0,365852.0,0.209076,203809.0,161469.0,0.262218,128068.0,111999.0,0.143474


In [223]:
market_trends_df.to_csv('/Users/paz/Documents/github/home-property-tax/wickersham/tax_year_2023/protest_evidence/market_trends.csv', index=False)

# The Neighborhood

In [300]:
neighborhood_query = '''
    select
        acct as account,
        protested,
        yr as year,
        neighborhood_code,
        lgl_2 as neighborhood,
        neighborhood_grp as neighborhood_group,
        map_facet,
        site_addr_1 as street_address,
        site_addr_3 as zipcode,
        tot_mkt_val as market_value,
        prior_tot_mkt_val as prior_market_value,
        tot_mkt_val - prior_tot_mkt_val - x_features_val as change_in_market_value,
        (tot_mkt_val - x_features_val) / (prior_tot_mkt_val) - 1 as pct_change_in_market_value
    from real_acct
    where
        true
        and state_class = 'A1'
        -- and coalesce(new_construction_val,0) < 100000
        and yr = '2023'
        and neighborhood_code in (7946, 7946.03, 7947)
'''

In [301]:
neighborhood_df = pd.read_sql_query(neighborhood_query, engine)

In [302]:
fig = px.histogram(neighborhood_df, x="pct_change_in_market_value", marginal="box", hover_data=neighborhood_df.columns)
fig.show()

In [307]:
neighborhood_df['pct_change_in_market_value'].quantile(.85)

0.09681595493719597

In [303]:
neighborhood_df.loc[neighborhood_df['account'].isin([zwickersham_acct, '931930000073'])]

,account,protested,year,Neighborhood_Code,neighborhood,neighborhood_group,map_facet,street_address,zipcode,market_value,prior_market_value,change_in_market_value,pct_change_in_market_value
51,931920000058,Y,2023,7947.0,BRIARGROVE PARK SEC 1,1117,4956D,10027 WICKERSHAM LN,77042,715818.0,652735.0,40846.0,0.062577
66,931930000073,N,2023,7947.0,BRIARGROVE PARK SEC 1,1117,4956B,10034 WICKERSHAM LN,77042,705486.0,662002.0,43484.0,0.065686


In [266]:
neighborhood_df.to_csv('/Users/paz/Documents/github/home-property-tax/wickersham/tax_year_2023/protest_evidence/neighborhood_2022_no_remodels.csv', index=False)

# Remodeling

In [285]:
remodeling_query = '''
    with cte_2022 as (
        select
            acct as account,
            yr as year,
            neighborhood_code,
            lgl_2 as neighborhood,
            neighborhood_grp as neighborhood_group,
            map_facet,
            site_addr_1 as street_address,
            site_addr_3 as zipcode,
            tot_mkt_val as market_value,
            tot_mkt_val - new_construction_val as market_value_less_new_construction,
            prior_tot_mkt_val as prior_market_value,
            new_construction_val as new_construction_value,
            tot_mkt_val - prior_tot_mkt_val as change_in_market_value,
            tot_mkt_val / prior_tot_mkt_val - 1 as pct_change_in_market_value,
            (tot_mkt_val - new_construction_val) / prior_tot_mkt_val - 1 as pct_change_in_market_less_new_construction,
            new_construction_val / prior_tot_mkt_val as new_construction_pct_total_prior_market_value,
            new_construction_val / (tot_mkt_val - prior_tot_mkt_val) as new_construction_pct_change_in_market_value
        from real_acct
        where
            true
            and state_class = 'A1'
            and new_construction_val >= 100000
            and neighborhood_code in (7946, 7946.03, 7947)
            and yr = '2022'
    ),
    cte_2023 as (
        SELECT 
            acct,
            tot_mkt_val
        from
            real_acct ra 
            inner join cte_2022 on cte_2022.account = ra.acct
        WHERE
            yr = '2023'
    )
    SELECT 
        cte_2022.*,
        cte_2023.tot_mkt_val as next_market_value,
        cte_2023.tot_mkt_val / cte_2022.market_value - 1 as next_market_value_pct_change
    from cte_2022
        left join cte_2023 on cte_2023.acct = cte_2022.account
'''

In [286]:
remodeling_df = pd.read_sql_query(remodeling_query, engine)

In [287]:
# remodeling_df_filtered = remodeling_df.loc[remodeling_df['neighborhood_code'].isin([7946, 7947, 7946.03])]
remodeling_df_filtered = remodeling_df.loc[remodeling_df['neighborhood_group'].isin([1117])]

In [288]:
fig = px.histogram(remodeling_df_filtered, x="pct_change_in_market_less_new_construction", marginal="box", hover_data=remodeling_df.columns)
fig.show()

In [309]:
remodeling_df_filtered.sort_values(by=['market_value'], ascending=True).reset_index(drop=True)

,account,year,neighborhood_code,neighborhood,neighborhood_group,map_facet,street_address,zipcode,market_value,market_value_less_new_construction,prior_market_value,new_construction_value,change_in_market_value,pct_change_in_market_value,pct_change_in_market_less_new_construction,new_construction_pct_total_prior_market_value,new_construction_pct_change_in_market_value,next_market_value,next_market_value_pct_change
0,932100000549,2022,7946.0,BRIARGROVE PARK SEC 1,1117,4957D,10015 VALLEY FORGE DR,77042,604893.0,424010.0,410153.0,180883.0,194740.0,0.474798,0.033785,0.441013,0.928844,1084691.0,0.793195
1,931920000058,2022,7947.0,BRIARGROVE PARK SEC 1,1117,4956D,10027 WICKERSHAM LN,77042,652735.0,477906.0,414458.0,174829.0,238277.0,0.574912,0.153087,0.421826,0.733722,715818.0,0.096644
2,931980000244,2022,7947.0,BRIARGROVE PARK SEC 1,1117,4956B,10019 OLYMPIA DR,77042,659910.0,550633.0,501501.0,109277.0,158409.0,0.315870,0.097970,0.217900,0.689841,702036.0,0.063836
3,931930000073,2022,7947.0,BRIARGROVE PARK SEC 1,1117,4956B,10034 WICKERSHAM LN,77042,662002.0,513605.0,481146.0,148397.0,180856.0,0.375886,0.067462,0.308424,0.820526,705486.0,0.065686
4,932100000533,2022,7946.0,BRIARGROVE PARK SEC 1,1117,4956B,10018 LONGMONT DR,77042,663879.0,410805.0,410805.0,253074.0,253074.0,0.616044,0.000000,0.616044,1.000000,1073233.0,0.616609
5,1075870001108,2022,7946.0,BRIARGROVE PARK SEC 6,1117,4956B,10219 CEDAR CREEK DR,77042,738300.0,573621.0,550396.0,164679.0,187904.0,0.341398,0.042197,0.299201,0.876400,762210.0,0.032385
6,982410000005,2022,7946.0,BRIARGROVE PARK SEC 3,1117,4957D,10102 LYNBROOK HOLLOW ST,77042,795866.0,666729.0,638717.0,129137.0,157149.0,0.246039,0.043857,0.202182,0.821749,818370.0,0.028276
7,932250000954,2022,7946.0,BRIARGROVE PARK SEC 1,1117,4957D,10219 SHADY RIVER DR,77042,814091.0,484403.0,657000.0,329688.0,157091.0,0.239104,-0.262705,0.501808,2.098707,1410472.0,0.732573
8,932200000821,2022,7946.0,BRIARGROVE PARK SEC 1,1117,4957D,10019 BRIAR DR,77042,841646.0,564457.0,437427.0,277189.0,404219.0,0.924083,0.290403,0.633681,0.685740,868062.0,0.031386
9,932140000673,2022,7946.0,BRIARGROVE PARK SEC 1,1117,4957D,10003 HOLLY SPRINGS DR,77042,847138.0,632839.0,629853.0,214299.0,217285.0,0.344977,0.004741,0.340237,0.986258,889442.0,0.049938


In [290]:
remodeling_df_filtered.to_csv('/Users/paz/Documents/github/home-property-tax/wickersham/tax_year_2023/protest_evidence/2022_remodels.csv', index=False)

# Comps

In [292]:
hcad_equity_comps = ['931980000216','931930000073','0931980000232','931980000235','931980000238']
hcad_sale_comps = ['931980000223','931990000274','931980000216','931940000110','932020000340']
zwicker_comps = ['931930000077','931940000096','931940000110','931980000240','932000000311']
subject = list(zwickersham_acct)
full_comp_list = hcad_equity_comps + hcad_sale_comps + zwicker_comps + subject
full_comp_tuple = tuple(full_comp_list)
hcad_equity_tuple = tuple(hcad_equity_comps)
hcad_sale_tuple = tuple(hcad_sale_comps)
zwicker_tuple = tuple(zwicker_comps)
subject_string = subject[0]

In [299]:
comp_query = f'''
    with cte_2022 as (
        select
            acct as account,
            case when acct in {hcad_equity_tuple} then true else false end as is_hcad_equity_comp,
            case when acct in {hcad_sale_tuple} then true else false end as is_hcad_sale_comp,
            case when acct in {zwicker_tuple} then true else false end as is_zwicker_comp,
            case when acct = '{subject_string}' then true else false end as is_subject,
            site_addr_1 as street_address,
            tot_mkt_val as market_value_2022,
            prior_tot_mkt_val as market_value_2021,
            new_construction_val as new_construction_value_2022,
            tot_mkt_val - prior_tot_mkt_val as change_in_market_value_2022,
            tot_mkt_val / prior_tot_mkt_val - 1 as pct_change_in_market_value_2022,
            new_construction_val / prior_tot_mkt_val as new_construction_pct_total_prior_market_value_2022,
            new_construction_val / (tot_mkt_val - prior_tot_mkt_val) as new_construction_pct_change_in_market_value_2022
        from real_acct
        where
            true
            and acct in {full_comp_tuple}
            and yr = '2022'
    ),
    cte_2023 as (
        SELECT 
            acct,
            tot_mkt_val
        from
            real_acct ra 
            inner join cte_2022 on cte_2022.account = ra.acct
        WHERE
            yr = '2023'
    )
    SELECT 
        cte_2022.*,
        cte_2023.tot_mkt_val as market_value_2023,
        cte_2023.tot_mkt_val / cte_2022.market_value_2022 - 1 as market_value_pct_change_2023
    from cte_2022
        left join cte_2023 on cte_2023.acct = cte_2022.account
'''

In [297]:
comp_df = pd.read_sql_query(comp_query, engine)

In [298]:
comp_df.sort_values(by=['market_value_pct_change_2023']).reset_index(drop=True)

,account,is_hcad_equity_comp,is_hcad_sale_comp,is_zwicker_comp,is_subject,street_address,market_value_2022,market_value_2021,new_construction_value_2022,change_in_market_value_2022,pct_change_in_market_value_2022,new_construction_pct_total_prior_market_value_2022,new_construction_pct_change_in_market_value_2022,market_value_2023,market_value_pct_change_2023
0,931930000073,1,0,0,0,10034 WICKERSHAM LN,705486.0,662002.0,0.0,43484.0,0.065686,0.000000,0.000000,705486.0,0.0
1,931930000077,0,0,1,0,10050 WICKERSHAM LN,600654.0,484889.0,86830.0,115765.0,0.238745,0.179072,0.750054,600654.0,0.0
2,931940000096,0,0,1,0,10022 OVERBROOK LN,585726.0,527773.0,0.0,57953.0,0.109807,0.000000,0.000000,585726.0,0.0
3,931940000110,0,1,1,0,10027 ELLA LEE LN,649335.0,567682.0,0.0,81653.0,0.143836,0.000000,0.000000,649335.0,0.0
4,931980000216,1,1,0,0,2203 BRIARPARK DR,624000.0,678852.0,0.0,-54852.0,-0.080801,0.000000,-0.000000,624000.0,0.0
5,931980000223,0,1,0,0,10010 MEADOW LAKE LN,668697.0,515514.0,133091.0,153183.0,0.297146,0.258171,0.868837,668697.0,0.0
6,931980000235,1,0,0,0,10055 OLYMPIA DR,644172.0,589338.0,0.0,54834.0,0.093043,0.000000,0.000000,644172.0,0.0
7,931980000238,1,0,0,0,10043 OLYMPIA DR,737634.0,575000.0,0.0,162634.0,0.282842,0.000000,0.000000,737634.0,0.0
8,931980000240,0,0,1,0,10035 OLYMPIA DR,541995.0,552187.0,0.0,-10192.0,-0.018458,0.000000,-0.000000,541995.0,0.0
9,931990000274,0,1,0,0,10030 OLYMPIA DR,707056.0,658092.0,0.0,48964.0,0.074403,0.000000,0.000000,707056.0,0.0
